# 1. Cleaning and preparing data:
   a. Clean the data of errors and inconsistencies.
   b. Deal with missing values and outliers.
   c. Transform the data into a format suitable for analysis and processing..


In [ ]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('/mnt/data/train_data.csv')
test_data = pd.read_csv('/mnt/data/test_data.csv')
test_data_hidden = pd.read_csv('/mnt/data/test_data_hidden.csv')

print(train_data.head())
print(test_data.head())
print(test_data_hidden.head())

# filling missing values with a placeholder for all columns
train_data.fillna("Unknown", inplace=True)
test_data.fillna("Unknown", inplace=True)
test_data_hidden.fillna("Unknown", inplace=True)

# converting 'reviews.date' to datetime format
train_data['reviews.date'] = pd.to_datetime(train_data['reviews.date'], errors='coerce')
test_data['reviews.date'] = pd.to_datetime(test_data['reviews.date'], errors='coerce')
test_data_hidden['reviews.date'] = pd.to_datetime(test_data_hidden['reviews.date'], errors='coerce')

# checking for data type inconsistencies 
print(train_data.info())
print(test_data.info())
print(test_data_hidden.info())

# saving the cleaned data
train_data.to_csv('/mnt/data/train_data_cleaned.csv', index=False)
test_data.to_csv('/mnt/data/test_data_cleaned.csv', index=False)
test_data_hidden.to_csv('/mnt/data/test_data_hidden_cleaned.csv', index=False)
